In [ ]:
!pip install pyspark
#Pyspark
from pyspark.sql import SQLContext, functions as f, SparkSession
#numpy
import numpy as np
#Spark Session
def SparkConfig():
  spark = SparkSession.builder.appName("Elephas_APP")\
  .config("spark.yarn.maxAppAttempts","2")\
  .config("spark.num.executors","50")\
  .config("spark.executor.memory","12g")\
  .config("spark.driver.memory", "12g")\
  .config("spark.memory.offHeap.enabled",True)\
  .config("spark.memory.offHeap.size","12g")\
  .getOrCreate()
  spark.sql("set spark.sql.legacy.timeParserPolicy")
  return spark
spark = SparkConfig()

     |████████████████████████████████| 212.4MB 69kB/s 
     |████████████████████████████████| 204kB 12.0MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=21c348eca6b158b5c3cc4d52c1be4ad373df1a26ff09df93a4089a1fc7f184e5
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [ ]:
#Process data
def Process(df):
  #Drop null values and errored "data.basic_info.locale" column 
  df = df.drop("data.basic_info.locale")
  df = df.na.drop(subset = ["label"])
  len = df.select("Field_3").count()

  #Birth month
  #df = df.filter(f.col("birth_month") in [np.arange(1,13)])
  # Get cols has timestamp, boolean type 
  timestamp_col = [c for c,t in df.dtypes if t == "timestamp"]
  bool_col = [c for c,t in df.dtypes if t == "boolean"]

  # Remove cols has timestamp type  and convert cols has boolean types to interger
  df = df.drop(*timestamp_col)
  for c in bool_col:
    df = df.withColumn(c,f.when(f.col(c)==True,1).when(f.col(c)==False,0).otherwise(2)) 
  # Drop column has null frequency greater than 0.9
  null_count = df.select([(f.count(f.when(f.isnan(c) | f.col(c).isNull(), c))).alias(c) for c in df.columns]).collect()
  null_drop = [c for c in df.columns if null_count[0][c] > 0.9*len]
  df = df.drop(*null_drop)

  # Fill miss value
  ## Fill miss value for "Field_82","birth_month" by -1  because it is category
  df = df.na.fill(-1,subset = ['Field_82','birth_month'])
  ## Fill miss value for string by "Unknown", numeric by "-9999....."
  df = df.na.fill("Unknown")
  df = df.replace("none","Unknown")
  df = df.replace("null","Unknown")
  df = df.replace("NaN","Unknown")
  df = df.na.fill(-999999999999999999999999999999999999999)
  return df


In [ ]:
#Load data
train = spark.read.format("csv").options(inferSchema = True,header = True,sep = ",").load("./Big Data/Data Kalapa/Dataset/preprocessed_train.csv")
train.show(5,False)

#Find number of rows, cols
num_rows = train.select("Field_3").count()
num_cols = len(train.columns)
print(f"NumOfRows before processing: {num_rows}")
print(f"NumOfColumns before processing: {num_cols}")

#Processing data
train = Process(train)
train.toPandas().to_csv("./Big Data/Data Kalapa/Dataset/Cleaned_Dataset3.csv",index = 0)

#Find number of rows, cols
num_rows = train.select("Field_3").count()
num_cols = len(train.columns)
print(f"NumOfRows after processing: {num_rows}")
print(f"NumOfColumns after processing: {num_cols}")

+-----+-------------------+-------------------+-------+-------+-------------------+-------------------+-------------------+-------------------+-------------------+--------+-------------------+--------+--------+-------------------+-----------------------------+--------+---------+--------+--------+--------+-------------------+--------+--------+--------+-------------------+-------------------+-------------------+-------------------+---------+--------+--------+--------+-------------------+--------+--------+-------------------+-------------------+--------+--------------------------------------------------+--------+---------+---------+--------+--------+--------+--------+--------+------------------------------+--------+--------+--------+------------------------+--------+--------+--------+--------+--------+---------+--------------+----------------------------+--------+--------+--------+-------------------+--------+--------+--------+--------+--------+-----------+---------------+----------------